In [1]:
import requests
import lxml.html as lh
import pandas as pd

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

page = requests.get(url)

doc = lh.fromstring(page.content)

tr_elements = doc.xpath('//tr')

In [3]:
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

Three columns per row


In [4]:
tr_elements = doc.xpath('//tr')

col=[]
i=0

for t in tr_elements[0]:
    i+=1
    name = t.text_content()
    print('%d:"%s"'%(i, name))
    col.append((name, []))

1:"Postcode"
2:"Borough"
3:"Neighbourhood
"


**Creating Pandas Dataframe**


In [6]:
for j in range(1, len(tr_elements)):
    T=tr_elements[j]
    
    if len(T)!=3:
        break
        
    i=0
    
    for t in T.iterchildren():
        data=t.text_content()
        
        if i>0:
            try:
                data=int(data)
            except:
                pass
        col[i][1].append(data)
        i+=1

In [7]:
[len(C) for (title, C) in col]

[287, 287, 287]

**Creating the Pandas Dataframe**

In [8]:
Dict ={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [10]:
df.columns=['Neighborhood', 'Postcode', 'Borough']

cols= df.columns.tolist()
cols

cols=cols[-1:]+cols[:-1]

df=df[cols]

df.head()

,Borough,Neighborhood,Postcode
0,Not assigned,Not assigned\n,M1A
1,Not assigned,Not assigned\n,M2A
2,North York,Parkwoods\n,M3A
3,North York,Victoria Village\n,M4A
4,Downtown Toronto,Harbourfront\n,M5A


**Cleaning messy strings**

In [11]:
df = df.replace('\n',' ', regex=True)
df.head()

,Borough,Neighborhood,Postcode
0,Not assigned,Not assigned,M1A
1,Not assigned,Not assigned,M2A
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,Harbourfront,M5A


**Dropping all cells with a Borough that is not assigned**

In [13]:
df.drop(df.index[df['Borough']=='Not assigned'], inplace =True)

df=df.reset_index(drop=True)

df.head(10)

,Borough,Neighborhood,Postcode
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,Harbourfront,M5A
3,North York,Lawrence Heights,M6A
4,North York,Lawrence Manor,M6A
5,Queen's Park,Not assigned,M7A
6,Downtown Toronto,Queen's Park,M9A
7,Scarborough,Rouge,M1B
8,Scarborough,Malvern,M1B
9,North York,Don Mills North,M3B


**Combining Neighborhoods based on similarPostcode and Borough**

In [14]:
df = df.groupby(['Postcode', 'Borough'])['Neighborhood'].apply(','.join).reset_index()
df.columns = ['Postcode', 'Borough', 'Neighborhood']
df.head(10)

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge ,Malvern"
1,M1C,Scarborough,"Highland Creek ,Rouge Hill ,Port Union"
2,M1E,Scarborough,"Guildwood ,Morningside ,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park ,Ionview ,Kennedy Park"
7,M1L,Scarborough,"Clairlea ,Golden Mile ,Oakridge"
8,M1M,Scarborough,"Cliffcrest ,Cliffside ,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff ,Cliffside West"


**Removing space in start of string**

In [15]:
df['Neighborhood'] = df['Neighborhood'].str.strip()

**Assigning Borough values to the neighborhood where value not assigned**

In [17]:
df.loc[df['Neighborhood'] == 'Not assigned', 'Neighborhood'] = df['Borough']
df[df['Borough'] == 'Queen\'s Park']

,Postcode,Borough,Neighborhood
85,M7A,Queen's Park,Queen's Park


In [18]:
df.shape

(103, 3)